# Exploratory Data Analysis Notebook

## What is Pneuomonia?

- Pneumonia is an infection that <b>inflames air sacs</b> in one or both lungs, which may fill with fluid.
- With pneumonia, the <b>air sacs may fill with fluid or pus.</b> 
- The infection can be <b>life-threatening</b> to anyone, but particularly to infants, children and people over 65.

## What Exactly are we Predicting?

- In this challenge, we are predicting whether pneumonia exists in a given image. 
- We do so by <b>predicting bounding boxes</b> around areas of the lung. 
- Samples <b>without</b> bounding boxes are <b>negative</b> and contain no definitive evidence of pneumonia. 
- Samples <b>with</b> bounding boxes <b>indicate evidence of pneumonia</b>.

## Format of Data

- <b><i>patientId_</i></b>- A patientId. Each patientId corresponds to a unique image.
- <b><i>x_</i></b> - the upper-left x coordinate of the bounding box.
- <b><i>y_</i></b> - the upper-left y coordinate of the bounding box.
- <b><i>width_</i></b> - the width of the bounding box.
- <b><i>height_</i></b> - the height of the bounding box.
- <b><i>Target_</i></b> - the binary Target, indicating whether this sample has evidence of pneumonia. (1 = true, 0 = false)